# Nosana GPU-Powered Jupyter Notebook Tutorial

## Introduction

Nosana is a GPU marketplace built to make AI compute accessible and affordable for businesses and developers. Focused on AI inference, Nosana connects Hosts—data centers and hardware owners—with Clients who need scalable and cost-effective GPU compute for AI Inference workloads.

This tutorial demonstrates how to deploy and interact with GPU-enabled Jupyter notebooks on the Nosana network using both the CLI and SDK approaches.

## What You'll Learn

- Understanding Nosana's job structure and GPU markets
- Deploying Jupyter notebooks with GPU support
- Monitoring and managing your deployments
- Interacting with your Jupyter instance
- Best practices for cost optimization

## Prerequisites

- Node.js (v18+)
- SOL tokens (minimum 0.05 SOL for transactions)
- NOS tokens for compute costs
- Basic understanding of Docker and Jupyter notebooks

## Core Concepts

### Nosana Job Structure

Each job on the Nosana network is essentially an AI inference task that runs from a Docker container. The jobs are designed to utilize GPU resources efficiently. Key components:

- **Operations (ops)**: Array of containerized tasks
- **Job States**: QUEUED → RUNNING → COMPLETED
- **Markets**: GPU groupings by type (3060, 3090, etc.)
- **Service URLs**: Access deployed services via unique endpoints

### GPU Markets

Nosana offers a variety of GPU markets tailored to different needs. Popular markets include:
- NVIDIA-3060: Budget-friendly option for light workloads
- NVIDIA-3090: High-performance for demanding tasks
- Custom markets for specific GPU requirements

## Method 1: Quick Start with CLI

The fastest way to deploy a Jupyter notebook is using the Nosana CLI.

### Installation & Setup

In [ ]:
# Install Nosana CLI globally
!npm install -g @nosana/cli

# Check wallet address and balances
!nosana address

### Job Definition

In [ ]:
# Create jupyter.json job definition
jupyter_job = {
    "version": "0.1",
    "type": "container",
    "meta": {
        "trigger": "cli"
    },
    "ops": [
        {
            "type": "container/run",
            "id": "jupyter-notebook",
            "args": {
                "cmd": [
                    "bash", "-c",
                    "source /etc/bash.bashrc && "
                    "jupyter notebook --notebook-dir=/tf "
                    "--ip 0.0.0.0 --no-browser --allow-root "
                    "--NotebookApp.token='' --NotebookApp.password=''"
                ],
                "expose": 8888,
                "image": "tensorflow/tensorflow:latest-gpu-jupyter",
                "gpu": True
            }
        }
    ]
}

# Save to file
import json
with open('jupyter.json', 'w') as f:
    json.dump(jupyter_job, f, indent=2)

print("Job definition created: jupyter.json")

### Deploy to Nosana Network

In [ ]:
# Deploy using CLI (choose appropriate market)
# For NVIDIA 3060 market (budget option):
!nosana job post --file jupyter.json --market nvidia-3060 --timeout 60

# Monitor job status
# Replace JOB_ID with actual job ID from deployment output
# !nosana job get JOB_ID --wait

## Method 2: Programmatic Control with SDK

For advanced users who need programmatic control over job lifecycle.

In [ ]:
# Install required packages
!npm install @nosana/sdk @solana/web3.js

In [ ]:
%%writefile deploy_jupyter.js
import { PublicKey } from '@solana/web3.js';
import { Client, sleep } from '@nosana/sdk';

// Job definition for GPU-enabled Jupyter
const jobDefinition = {
    "version": "0.1",
    "type": "container",
    "meta": { "trigger": "sdk" },
    "ops": [{
        "type": "container/run",
        "id": "jupyter-gpu",
        "args": {
            "cmd": [
                "bash", "-c",
                "pip install matplotlib seaborn scikit-learn && "
                "jupyter notebook --notebook-dir=/tf --ip 0.0.0.0 "
                "--no-browser --allow-root --NotebookApp.token='' "
                "--NotebookApp.password=''"
            ],
            "expose": 8888,
            "image": "tensorflow/tensorflow:2.17.0-gpu-jupyter",
            "gpu": true,
            "env": {
                "CUDA_VISIBLE_DEVICES": "0"
            }
        }
    }]
};

async function deployJupyter() {
    // Initialize client with private key
    const privateKey = process.env.SOLANA_KEY || 'your_private_key_here';
    const nosana = new Client('mainnet', privateKey);
    
    // Check balances
    console.log(`Wallet: ${nosana.solana.wallet.publicKey.toString()}`);
    console.log(`SOL: ${await nosana.solana.getSolBalance()} SOL`);
    console.log(`NOS: ${(await nosana.solana.getNosBalance())?.amount.toString()} NOS`);
    
    // Upload job to IPFS
    const ipfsHash = await nosana.ipfs.pin(jobDefinition);
    console.log(`IPFS Hash: ${ipfsHash}`);
    
    // Post to GPU market
    const market = new PublicKey('97G9NnvBDQ2WpKu6fasoMsAKmfj63C9rhysJnkeWodAf');
    const response = await nosana.jobs.list(ipfsHash, market);
    
    console.log(`\nDeployment Details:`);
    console.log(`Job ID: ${response.job}`);
    console.log(`Service URL: https://${response.job}.node.k8s.prd.nos.ci`);
    console.log(`Explorer: https://dashboard.nosana.com/jobs/${response.job}`);
    
    // Monitor job progress
    let job = await nosana.jobs.get(response.job);
    console.log(`\nMonitoring job progress...`);
    
    while (!job || job.state !== "RUNNING") {
        job = await nosana.jobs.get(response.job);
        console.log(`Status: ${job.state}`);
        if (job.state === "RUNNING") break;
        await sleep(10);
    }
    
    console.log(`\n✅ Jupyter notebook is now running!`);
    console.log(`Access at: https://${response.job}.node.k8s.prd.nos.ci`);
    
    return response.job;
}

// Run deployment
deployJupyter().catch(console.error);

In [ ]:
# Execute the deployment script
!node deploy_jupyter.js

## Interacting with Your Jupyter Instance

Once deployed, your Jupyter notebook will be accessible via the service URL. The service will be exposed at https://[JOB_ID].node.k8s.prd.nos.ci

In [ ]:
# Example: Test GPU availability in your deployed notebook
gpu_test_code = '''
import tensorflow as tf
import torch

# Check TensorFlow GPU
print("TensorFlow version:", tf.__version__)
print("GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

# Check PyTorch GPU
print("PyTorch CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("GPU Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
'''

print("Copy this code to test GPU in your deployed Jupyter notebook:")
print(gpu_test_code)

## Job Management

### Monitoring and Extending Jobs

In [ ]:
%%writefile manage_job.js
import { Client } from '@nosana/sdk';

class JobManager {
    constructor(privateKey) {
        this.nosana = new Client('mainnet', privateKey);
    }
    
    async getJobStatus(jobId) {
        const job = await this.nosana.jobs.get(jobId);
        return {
            state: job.state,
            startTime: job.startTime,
            duration: job.duration,
            price: job.price
        };
    }
    
    async extendJob(jobId, additionalMinutes) {
        console.log(`Extending job ${jobId} by ${additionalMinutes} minutes`);
        // Use CLI for extension
        const { exec } = require('child_process');
        return new Promise((resolve, reject) => {
            exec(`nosana job extend ${jobId} --timeout ${additionalMinutes}`, 
                (error, stdout, stderr) => {
                    if (error) reject(error);
                    else resolve(stdout);
                });
        });
    }
    
    async stopJob(jobId) {
        console.log(`Stopping job ${jobId}`);
        const { exec } = require('child_process');
        return new Promise((resolve, reject) => {
            exec(`nosana job stop ${jobId}`, (error, stdout, stderr) => {
                if (error) reject(error);
                else resolve(stdout);
            });
        });
    }
}

// Usage example
const manager = new JobManager(process.env.SOLANA_KEY);
// const status = await manager.getJobStatus('YOUR_JOB_ID');
// console.log('Job Status:', status);

## Cost Optimization Tips

1. **Choose the Right Market**: For real-time updates on prices, job timeouts, queue lengths, and more, visit the Nosana explorer.

2. **Monitor Usage**: Set appropriate timeouts to avoid unnecessary costs

3. **Resource Planning**: Use lower-tier GPUs for development, higher-tier for production

4. **Batch Processing**: Combine multiple tasks in single deployments when possible

In [ ]:
# Example: Market comparison for cost optimization
markets_comparison = {
    "nvidia-3060": {
        "price_per_second": 0.000043,
        "hourly_cost": 0.000043 * 3600,
        "use_cases": ["Development", "Light ML tasks", "Testing"]
    },
    "nvidia-3090": {
        "price_per_second": 0.000115,
        "hourly_cost": 0.000115 * 3600,
        "use_cases": ["Heavy ML training", "Large models", "Production"]
    }
}

print("Market Comparison for Cost Optimization:")
for market, details in markets_comparison.items():
    print(f"\n{market}:")
    print(f"  Hourly Cost: {details['hourly_cost']:.4f} NOS")
    print(f"  Best for: {', '.join(details['use_cases'])}")

## Best Practices & Troubleshooting

### Common Issues

1. **Insufficient Funds**: You will need a minimum of 0.05 SOL, and a minimum amount of NOS to pay for the Nosana deployment.

2. **Job Timeout**: Set realistic timeouts based on your workflow requirements

3. **Image Selection**: Use official images when possible for better reliability

### Security Considerations

- Never hardcode private keys in notebooks
- Use environment variables for sensitive data
- Monitor job costs regularly
- Stop unused deployments promptly

## Conclusion

You've learned how to deploy GPU-enabled Jupyter notebooks on Nosana using both CLI and SDK approaches. Nosana significantly streamlines the process of setting up high-performance AI inference services. Start with the CLI for quick experimentation, then move to the SDK for production workflows requiring programmatic control.

### Next Steps

- Explore the [Nosana Dashboard](https://dashboard.nosana.com) for visual job management
- Try different GPU markets based on your performance needs
- Integrate Nosana deployments into your ML workflows
- Join the [Nosana Discord](https://discord.gg/nosana) community for support